In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
## Add your information in here.
NAME = "Piyush Bagad"
NAME2 = ""
EMAIL = "piyush.bagad@student.uva.nl"
EMAIL2 = ""

In [3]:
# Necessary libraries that will be employed
import itertools
from copy import deepcopy
from dataclasses import dataclass
from typing import List

from sudoku_func import *

#!{sys.executable} -m pip install clingo
#!{sys.executable} -m pip install python-sat
import clingo
import pysat
from pysat.formula import CNF
from pysat.solvers import MinisatGH

In [48]:
def solve_sudoku_ASP(sudoku, k):
    """
    Solve a sudoku input by encoding the problem into ASP, calling an ASP solver,
    and retrieving the solution for the sudoku input from an answer set given by the ASP solver.

    Parameters:
        sudoku (list(list(int))): An input puzzle.
        k (int): The dimension of the sudoku input.

    Returns:
        A generator using yield instead of returning a (list(list(list(int)))).
    """

    
    constant_definition = f"        #const k={k}.\n"
    
    program = constant_definition + \
    """
        % define variables
        row(1..k*k).
        col(1..k*k).
        value(1..k*k).

        % define veriable to denote a cell in the sudoku
        cell(I, J) :- row(I), col(J).

        % every cell should be assigned exactly 1 value
        1 { assign(I, J, V): value(V) } 1 :- cell(I, J).

        % every row should have different values
        :- assign(I, J, V), assign(I', J, V), I!=I'.

        % every column should have different values
        :- assign(I, J, V), assign(I, J', V), J!=J'.

        % no two values in a single *block* should be the same
        % note that if (I, J) and (I', J') are two different cells
        % then they lie in the same block iff (I - 1) / 3 == (I' - 1) / 3
        % and likewise for the column
        :- assign(I, J, V), assign(I', J', V), J!=J', I!=I', (I - 1)/3==(I' - 1)/3, (J - 1)/3==(J' - 1)/3.

        % define a predicate that takes in (I, J, I', J') and
        % returns true iff (I, J) and (I', J') are `adjacent_neighbors`
        same_cell(I, J, I', J') :- cell(I, J), cell(I', J'), I==I', J==J'.
        row_diff(I, I') :- row(I), row(I'), (I - I') <= 1, (I - I') >= -1.
        col_diff(I, I') :- col(I), col(I'), (I - I') <= 1, (I - I') >= -1.
        adjacent_neighbors(I, J, I', J') :- cell(I,J), cell(I',J'), row_diff(I, I'), col_diff(J, J'), not same_cell(I, J, I', J').
        :- assign(I, J, V), assign(I', J', V), adjacent_neighbors(I, J, I', J').

        % knights constraint
        valid_cell(I, J):- cell(I, J), I >= 1, J <= k*k.
        % decides if cell (I, J) is attacked by a knights move by cell (I', J')
        knight_attacked(I, J, I', J') :- cell(I, J), cell(I', J'), I' - I >= -2, I' - I <= 2, I!=I', J' - J >= -2, J' - J <= 2, J!=J', I' - I != J'-J, I' - I != J - J'.
        :- assign(I, J, V), assign(I', J', V), valid_cell(I, J), valid_cell(I', J'), knight_attacked(I, J, I', J').

        % input constraint
    """

    for I in range(len(sudoku)):
        for J in range(len(sudoku)):
            V = sudoku[I][J]
            if V:
                program += f"    :- not assign({I + 1}, {J + 1}, {V}).\n"
    
    print(program)

    control = clingo.Control()
    control.add("base", [], program)
    control.ground([("base", [])])

    control.configuration.solve.models = 0

    def filter_answer_set(model, convert_to_str=False, verbose=True):

        # get all assignments
        assignments = [atom for atom in model.symbols(atoms=True) if str(atom).startswith("assign")]

        # get all assignments that are positive
        assignments = [atom for atom in assignments if atom.positive]

        if convert_to_str:
            assignments = [str(atom) for atom in assignments]
        else:
            solved_sudoku = deepcopy(sudoku)
            for atom in assignments:
                arguments = atom.arguments
                i, j, v = [x.number for x in arguments]
                solved_sudoku[i - 1][j - 1] = v

            if verbose:
                print("Input:")
                print(pretty_repr(sudoku, k=k))
                print()
                print("Solution:")
                print(pretty_repr(solved_sudoku, k=k))
                print()

        return solved_sudoku

    def generate():
        for model in control.solve(yield_=True):
            solved_sudoku = filter_answer_set(model, convert_to_str=False, verbose=False)
            yield solved_sudoku

    solutions = generate()
        
    
    return solutions


In [49]:
test_input = test_inputs[4]

assert check_num_solutions(
    test_input.sudoku,
    test_input.k,
    test_input.num_solutions,
    solve_sudoku_ASP
) == True


        #const k=4.

        % define variables
        row(1..k*k).
        col(1..k*k).
        value(1..k*k).

        % define veriable to denote a cell in the sudoku
        cell(I, J) :- row(I), col(J).

        % every cell should be assigned exactly 1 value
        1 { assign(I, J, V): value(V) } 1 :- cell(I, J).

        % every row should have different values
        :- assign(I, J, V), assign(I', J, V), I!=I'.

        % every column should have different values
        :- assign(I, J, V), assign(I, J', V), J!=J'.

        % no two values in a single *block* should be the same
        % note that if (I, J) and (I', J') are two different cells
        % then they lie in the same block iff (I - 1) / 3 == (I' - 1) / 3
        % and likewise for the column
        :- assign(I, J, V), assign(I', J', V), J!=J', I!=I', (I - 1)/3==(I' - 1)/3, (J - 1)/3==(J' - 1)/3.

        % define a predicate that takes in (I, J, I', J') and
        % returns true iff (I, J) and (I', J') are `adj

AssertionError: 

### Debug

In [50]:
def filter_answer_set(sudoku, k, model, convert_to_str=False, verbose=True):

        # get all assignments
        assignments = [atom for atom in model.symbols(atoms=True) if str(atom).startswith("assign")]

        # get all assignments that are positive
        assignments = [atom for atom in assignments if atom.positive]

        if convert_to_str:
            assignments = [str(atom) for atom in assignments]
        else:
            solved_sudoku = deepcopy(sudoku)
            for atom in assignments:
                arguments = atom.arguments
                i, j, v = [x.number for x in arguments]
                solved_sudoku[i - 1][j - 1] = v

            if verbose:
                print("Input:")
                print(pretty_repr(sudoku, k=k))
                print()
                print("Solution:")
                print(pretty_repr(solved_sudoku, k=k))
                print()

        return solved_sudoku


def print_answer_set_sudoku(sudoku, k, program):
    control = clingo.Control()
    control.add("base", [], program)
    control.ground([("base", [])])

    control.configuration.solve.models = 0

    def generate():
        for model in control.solve(yield_=True):
            solved_sudoku = filter_answer_set(sudoku, k, model, convert_to_str=False, verbose=True)
            yield solved_sudoku

    solutions = generate()
    
#     sample_solution = next(solutions)
    
#     return sample_solution
    
    return solutions


In [66]:
sudoku, k = test_input.sudoku, test_input.k

program = """
    #const k=4.

    % define variables
    row(1..k*k).
    col(1..k*k).
    value(1..k*k).

    % define veriable to denote a cell in the sudoku
    cell(I, J) :- row(I), col(J).

    % every cell should be assigned exactly 1 value
    1 { assign(I, J, V): value(V) } 1 :- cell(I, J).

    % every row should have different values
    :- assign(I, J, V), assign(I', J, V), I!=I'.

    % every column should have different values
    :- assign(I, J, V), assign(I, J', V), J!=J'.

    % no two values in a single *block* should be the same
    % note that if (I, J) and (I', J') are two different cells
    % then they lie in the same block iff (I - 1) / k == (I' - 1) / k
    % and likewise for the column
    :- assign(I, J, V), assign(I', J', V), J!=J', I!=I', (I - 1)/k==(I' - 1)/k, (J - 1)/k==(J' - 1)/k.

    % define a predicate that takes in (I, J, I', J') and
    % returns true iff (I, J) and (I', J') are `adjacent_neighbors`
    same_cell(I, J, I', J') :- cell(I, J), cell(I', J'), I==I', J==J'.
    row_diff(I, I') :- row(I), row(I'), (I - I') <= 1, (I - I') >= -1.
    col_diff(I, I') :- col(I), col(I'), (I - I') <= 1, (I - I') >= -1.
    adjacent_neighbors(I, J, I', J') :- cell(I,J), cell(I',J'), row_diff(I, I'), col_diff(J, J'), not same_cell(I, J, I', J').
    :- assign(I, J, V), assign(I', J', V), adjacent_neighbors(I, J, I', J').

    % knights constraint
    valid_cell(I, J):- cell(I, J), I >= 1, J <= k*k.
    % decides if cell (I, J) is attacked by a knights move by cell (I', J')
    knight_attacked(I, J, I', J') :- cell(I, J), cell(I', J'), I' - I >= -2, I' - I <= 2, I!=I', J' - J >= -2, J' - J <= 2, J!=J', I' - I != J'-J, I' - I != J - J'.
    :- assign(I, J, V), assign(I', J', V), valid_cell(I, J), valid_cell(I', J'), knight_attacked(I, J, I', J').

    % input constraint
"""
for I in range(len(sudoku)):
    for J in range(len(sudoku)):
        V = sudoku[I][J]
        if V:
            program += f"    :- not assign({I + 1}, {J + 1}, {V}).\n"


In [67]:
print(program)


    #const k=4.

    % define variables
    row(1..k*k).
    col(1..k*k).
    value(1..k*k).

    % define veriable to denote a cell in the sudoku
    cell(I, J) :- row(I), col(J).

    % every cell should be assigned exactly 1 value
    1 { assign(I, J, V): value(V) } 1 :- cell(I, J).

    % every row should have different values
    :- assign(I, J, V), assign(I', J, V), I!=I'.

    % every column should have different values
    :- assign(I, J, V), assign(I, J', V), J!=J'.

    % no two values in a single *block* should be the same
    % note that if (I, J) and (I', J') are two different cells
    % then they lie in the same block iff (I - 1) / k == (I' - 1) / k
    % and likewise for the column
    :- assign(I, J, V), assign(I', J', V), J!=J', I!=I', (I - 1)/k==(I' - 1)/k, (J - 1)/k==(J' - 1)/k.

    % define a predicate that takes in (I, J, I', J') and
    % returns true iff (I, J) and (I', J') are `adjacent_neighbors`
    same_cell(I, J, I', J') :- cell(I, J), cell(I', J'), I==I'

In [68]:
solutions = print_answer_set_sudoku(sudoku, k, program)

In [69]:
sample_solution = next(solutions)
check_solution(sudoku, k, sample_solution)


Input:
.-------------.-------------.-------------.-------------.
| 03 01 15 13 | 07 16 05 12 | 04 02 14 10 | 08 09 06    | 
| 06 04 05 08 | 09 11 02 10 | 16 15 07 03 | 01 14 13    | 
| 10 07 14 02 | 15 06 01 13 | 09 11 08 12 | 04 16 03    | 
| 12 16 09 11 | 03 14 04 08 | 06 05 01 13 | 02 15 07    | 
.-------------.-------------.-------------.-------------.
| 04 02 10 01 | 16 09 12    | 03 07 15 14 | 05 06 08    | 
| 09 08 03 06 | 04 05 13    | 01 16 12 11 | 07 10 02    | 
| 15 13 12 14 | 02 08 07    | 05 10 04 09 | 03 01 11    | 
| 11 05 16 07 | 10 01 14    | 02 08 13 06 | 12 04 09    | 
.-------------.-------------.-------------.-------------.
| 08 10 02 03 | 11 04 15    | 12 01 05 07 | 16 13 14    | 
| 16 09 13 15 | 08 12 06    | 14 03 10 02 | 11 05 01    | 
| 07 14 11    | 01 02 10    | 15 13 06 04 | 09 08       | 
| 01 12 06    | 14 13 03    | 08 09 11 16 | 15 02       | 
.-------------.-------------.-------------.-------------.
| 13 03 07    | 12 15 09    | 10 14 02 01 | 06 11    

True